In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from pytorch_tabnet.tab_model import TabNetClassifier
import torch

# Thiết lập seed để kết quả có thể tái tạo
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# 1. Tải và khám phá dữ liệu
# Giả sử bạn có dữ liệu gian lận, thay đổi đường dẫn thích hợp
# Ví dụ: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
df = pd.read_csv('creditcard.csv')

print(f"Kích thước dữ liệu: {df.shape}")
print("\nThông tin dữ liệu:")
print(df.info())

print("\nKiểm tra giá trị thiếu:")
print(df.isnull().sum())

print("\nThống kê mô tả:")
print(df.describe())

# Kiểm tra mất cân bằng dữ liệu
print("\nPhân phối nhãn:")
print(df['Class'].value_counts())
print(f"Tỷ lệ gian lận: {df['Class'].mean()*100:.2f}%")

# 2. Tiền xử lý dữ liệu
# Chia tập dữ liệu
X = df.drop('Class', axis=1)
y = df['Class']

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Chia tập dữ liệu
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=RANDOM_SEED, stratify=y
)
print(f"\nKích thước tập huấn luyện: {X_train.shape}")
print(f"Kích thước tập kiểm tra: {X_test.shape}")

# 3. Tạo và huấn luyện mô hình TabNet
# Khởi tạo mô hình TabNet với các tham số
tabnet_params = {
    "n_d": 64,  # Chiều của decision prediction layer
    "n_a": 64,  # Chiều của attention embedding
    "n_steps": 5,  # Số bước trong mạng
    "gamma": 1.5,  # Tham số kiểm soát sự dùng lại feature
    "n_independent": 2,  # Số bước độc lập
    "n_shared": 2,  # Số bước chia sẻ
    "lambda_sparse": 1e-3,  # Tham số điều chỉnh sparse regularization
    "optimizer_fn": torch.optim.Adam,
    "optimizer_params": dict(lr=2e-2),
    "mask_type": "entmax",  # Loại mask để sử dụng
    "scheduler_params": dict(
        mode="min", patience=5, min_lr=1e-5, factor=0.5,
    ),
    "scheduler_fn": torch.optim.lr_scheduler.ReduceLROnPlateau,
    "seed": RANDOM_SEED,
    "verbose": 1
}

# Khởi tạo mô hình
model = TabNetClassifier(**tabnet_params)

# Huấn luyện mô hình
model.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    eval_name=['train', 'test'],
    eval_metric=['auc'],
    max_epochs=50,
    patience=10,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

# 4. Đánh giá mô hình
# Dự đoán
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Báo cáo phân loại
print("\nBáo cáo phân loại:")
print(classification_report(y_test, y_pred))

# Ma trận nhầm lẫn
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Bình thường', 'Gian lận'], 
            yticklabels=['Bình thường', 'Gian lận'])
plt.xlabel('Dự đoán')
plt.ylabel('Thực tế')
plt.title('Ma trận nhầm lẫn')
plt.show()

# Đường cong ROC
auc_score = roc_auc_score(y_test, y_pred_proba)
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'TabNet (AUC = {auc_score:.3f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Đường cong ROC')
plt.legend()
plt.show()

# 5. Phân tích đặc trưng quan trọng
# TabNet có thể cung cấp thông tin về đặc trưng quan trọng
feature_importances = model.feature_importances_
feature_names = X.columns

# Hiển thị top 15 đặc trưng quan trọng nhất
plt.figure(figsize=(10, 8))
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
})
importance_df = importance_df.sort_values('Importance', ascending=False).head(15)
sns.barplot(x='Importance', y='Feature', data=importance_df)
plt.title('Top 15 đặc trưng quan trọng nhất')
plt.tight_layout()
plt.show()

# 6. Giải thích mô hình cho từng dự đoán
# TabNet cho phép giải thích từng dự đoán cụ thể
explain_matrix, masks = model.explain(X_test)

# Hiển thị sự giải thích cho một giao dịch gian lận cụ thể
fraud_indices = np.where(y_test == 1)[0]
if len(fraud_indices) > 0:
    fraud_idx = fraud_indices[0]  # Lấy một giao dịch gian lận đầu tiên
    
    plt.figure(figsize=(14, 5))
    plt.subplot(1, 2, 1)
    plt.bar(range(len(feature_names)), X_test[fraud_idx])
    plt.xticks(range(len(feature_names)), feature_names, rotation=90)
    plt.title('Giá trị đặc trưng của giao dịch gian lận')
    
    plt.subplot(1, 2, 2)
    plt.bar(range(len(feature_names)), explain_matrix[fraud_idx])
    plt.xticks(range(len(feature_names)), feature_names, rotation=90)
    plt.title('Mức độ ảnh hưởng của đặc trưng đối với dự đoán')
    plt.tight_layout()
    plt.show()

print("\nMô hình TabNet đã được huấn luyện và đánh giá thành công cho bài toán phát hiện gian lận!")

In [ ]:
# Định nghĩa các hàm đánh giá mô hình
def evaluate_model(model, X, y, threshold=0.5, dataset_name="Unknown"):
    """Đánh giá mô hình và trả về các metric cơ bản"""
    # Dự đoán xác suất
    y_pred_proba = model.predict_proba(X)[:, 1]

    # Tính AUC và Average Precision
    auc = roc_auc_score(y, y_pred_proba)
    avg_precision = average_precision_score(y, y_pred_proba)

    # Chuyển xác suất thành nhãn dựa vào ngưỡng
    y_pred = (y_pred_proba >= threshold).astype(int)

    # Tạo báo cáo phân loại
    class_report = classification_report(y, y_pred, output_dict=True)

    # Tạo ma trận nhầm lẫn
    conf_matrix = confusion_matrix(y, y_pred)

    print(f"\n--- Kết quả đánh giá trên tập {dataset_name} ---")
    print(f"AUC: {auc:.4f}")
    print(f"Average Precision: {avg_precision:.4f}")
    print(f"Precision (Fraud): {class_report['1']['precision']:.4f}")
    print(f"Recall (Fraud): {class_report['1']['recall']:.4f}")
    print(f"F1-score (Fraud): {class_report['1']['f1-score']:.4f}")

    return {
        'auc': auc,
        'avg_precision': avg_precision,
        'precision': class_report['1']['precision'],
        'recall': class_report['1']['recall'],
        'f1': class_report['1']['f1-score'],
        'y_pred_proba': y_pred_proba,
        'conf_matrix': conf_matrix
    }


def plot_confusion_matrix(conf_matrix, title="Ma trận nhầm lẫn"):
    """Vẽ ma trận nhầm lẫn"""
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Bình thường', 'Gian lận'],
                yticklabels=['Bình thường', 'Gian lận'])
    plt.xlabel('Dự đoán')
    plt.ylabel('Thực tế')
    plt.title(title)
    plt.show()


def plot_pr_curve(y_true, y_score, title="Precision-Recall Curve"):
    """Vẽ đường cong Precision-Recall"""
    precision, recall, thresholds = precision_recall_curve(y_true, y_score)

    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, label=f'AP={average_precision_score(y_true, y_score):.3f}')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()


# Định nghĩa hàm objective cho Optuna
def objective(trial):
    """Hàm mục tiêu cho Optuna để tối ưu hóa siêu tham số"""
    # Định nghĩa không gian tìm kiếm siêu tham số
    tabnet_params = {
        "n_d": trial.suggest_int("n_d", 8, 64),
        "n_a": trial.suggest_int("n_a", 8, 64),
        "n_steps": trial.suggest_int("n_steps", 3, 10),
        "gamma": trial.suggest_float("gamma", 1.0, 2.0),
        "n_independent": trial.suggest_int("n_independent", 1, 5),
        "n_shared": trial.suggest_int("n_shared", 1, 5),
        "lambda_sparse": trial.suggest_float("lambda_sparse", 1e-6, 1e-2, log=True),
        "optimizer_fn": torch.optim.Adam,
        "optimizer_params": {
            "lr": trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True),
        },
        "mask_type": trial.suggest_categorical("mask_type", ["sparsemax", "entmax"]),
        "momentum": trial.suggest_float("momentum", 0.01, 0.4),
        "scheduler_params": {
            "mode": "max",
            "patience": trial.suggest_int("scheduler_patience", 3, 10),
            "min_lr": 1e-5,
            "factor": trial.suggest_float("scheduler_factor", 0.1, 0.5),
        },
        "scheduler_fn": torch.optim.lr_scheduler.ReduceLROnPlateau,
        "verbose": 0,
        "seed": 42
    }

    # Tham số huấn luyện
    max_epochs = trial.suggest_int("max_epochs", 30, 100)
    patience = trial.suggest_int("patience", 5, 15)
    batch_size = trial.suggest_categorical("batch_size", [512, 1024, 2048, 4096])
    virtual_batch_size = trial.suggest_categorical("virtual_batch_size", [64, 128, 256, 512])

    # Khởi tạo mô hình TabNet
    model = TabNetClassifier(**tabnet_params)

    # Try-except để xử lý các lỗi có thể xảy ra trong quá trình huấn luyện
    try:
        # Huấn luyện mô hình
        model.fit(
            X_train=X_train, y_train=y_train,
            eval_set=[(X_train, y_train), (X_val, y_val)],
            eval_name=['train', 'val'],
            eval_metric=['auc'],
            max_epochs=max_epochs,
            patience=patience,
            batch_size=batch_size,
            virtual_batch_size=virtual_batch_size,
            num_workers=0,
            drop_last=False
        )

        # Dự đoán trên tập validation
        y_pred_proba = model.predict_proba(X_val)[:, 1]
        val_auc = roc_auc_score(y_val, y_pred_proba)
        val_ap = average_precision_score(y_val, y_pred_proba)

        # Lưu giá trị tốt nhất của AUC vào thử nghiệm
        trial.set_user_attr("best_epoch", model.best_epoch)
        trial.set_user_attr("val_ap", val_ap)

        print(f"Trial #{trial.number}: val_auc = {val_auc:.4f}, val_ap = {val_ap:.4f}, "
              f"best_epoch = {model.best_epoch}, n_d = {tabnet_params['n_d']}, "
              f"n_a = {tabnet_params['n_a']}, batch_size = {batch_size}")

        return val_auc

    except Exception as e:
        print(f"Lỗi trong trial #{trial.number}: {e}")
        return float('-inf')  # Trả về giá trị thấp nếu có lỗi


# Chạy tối ưu hóa siêu tham số
# Thiết lập seed
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# Tạo một nghiên cứu Optuna mới
study_name = "tabnet_fraud_detection"
storage_name = "sqlite:///tabnet_fraud_detection.db"  # Lưu kết quả vào SQLite DB

# Kiểm tra xem đã có nghiên cứu cũ chưa
try:
    study = optuna.create_study(
        study_name=study_name,
        storage=storage_name,
        load_if_exists=True,
        direction="maximize"
    )
    print(f"Đã tải nghiên cứu hiện có với {len(study.trials)} trials đã hoàn thành.")
except:
    study = optuna.create_study(
        study_name=study_name,
        storage=storage_name,
        direction="maximize"
    )
    print("Đã tạo nghiên cứu mới.")

# Số lượng thử nghiệm và thời gian tối đa
n_trials = 30  # Điều chỉnh số lượng trials theo nhu cầu
timeout = 3600 * 4  # Tối đa 4 giờ

print(f"Bắt đầu tối ưu hóa siêu tham số với {n_trials} trials...")
start_time = time.time()

# Chạy tối ưu hóa
study.optimize(objective, n_trials=n_trials, timeout=timeout)

end_time = time.time()
duration = end_time - start_time
print(f"\nĐã hoàn thành tối ưu hóa trong {duration / 60:.2f} phút.")
# Hiển thị kết quả tối ưu hóa
# Lấy siêu tham số tốt nhất
best_params = study.best_params
best_trial = study.best_trial
best_value = study.best_value

print("\n========= Kết quả tối ưu hóa siêu tham số =========")
print(f"Best Trial: #{best_trial.number}")
print(f"Best AUC: {best_value:.4f}")
print(f"Best Average Precision: {best_trial.user_attrs['val_ap']:.4f}")
print(f"Best Epoch: {best_trial.user_attrs['best_epoch']}")

print("\nSiêu tham số tốt nhất:")
for param, value in best_params.items():
    print(f"- {param}: {value}")
# Tạo mô hình cuối cùng với siêu tham số tốt nhất
print("Huấn luyện mô hình cuối cùng với siêu tham số tốt nhất...")

# Khởi tạo tham số từ kết quả tối ưu hóa
final_tabnet_params = {
    "n_d": best_params["n_d"],
    "n_a": best_params["n_a"],
    "n_steps": best_params["n_steps"],
    "gamma": best_params["gamma"],
    "n_independent": best_params["n_independent"],
    "n_shared": best_params["n_shared"],
    "lambda_sparse": best_params["lambda_sparse"],
    "optimizer_fn": torch.optim.Adam,
    "optimizer_params": {
        "lr": best_params["learning_rate"],
    },
    "mask_type": best_params["mask_type"],
    "momentum": best_params["momentum"],
    "scheduler_params": {
        "mode": "max",
        "patience": best_params["scheduler_patience"],
        "min_lr": 1e-5,
        "factor": best_params["scheduler_factor"],
    },
    "scheduler_fn": torch.optim.lr_scheduler.ReduceLROnPlateau,
    "verbose": 1,
    "seed": 42
}

# Huấn luyện mô hình cuối cùng trên tập huấn luyện + validation
final_model = TabNetClassifier(**final_tabnet_params)
final_model.fit(
    X_train=X_train_val, y_train=y_train_val,
    eval_set=[(X_train_val, y_train_val), (X_test, y_test)],
    eval_name=['train_val', 'test'],
    eval_metric=['auc'],
    max_epochs=best_params["max_epochs"],
    patience=best_params["patience"],
    batch_size=best_params["batch_size"],
    virtual_batch_size=best_params["virtual_batch_size"],
    num_workers=0,
    drop_last=False
)

# Đánh giá mô hình cuối cùng
# Tìm ngưỡng tối ưu dựa trên tập validation
y_val_proba = final_model.predict_proba(X_val)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_val, y_val_proba)

# Tính F1 cho mỗi ngưỡng
f1_scores = 2 * recall * precision / (recall + precision + 1e-10)
optimal_threshold_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_threshold_idx]

print(f"Ngưỡng phân loại tối ưu: {optimal_threshold:.4f}")

# Đánh giá trên tập test với ngưỡng tối ưu
test_results = evaluate_model(final_model, X_test, y_test,
                              threshold=optimal_threshold,
                              dataset_name="Test")

# Vẽ ma trận nhầm lẫn
plot_confusion_matrix(test_results['conf_matrix'], "Ma trận nhầm lẫn - Tập Test")

# Vẽ đường cong Precision-Recall
plot_pr_curve(y_test, test_results['y_pred_proba'], "Precision-Recall Curve - Tập Test")
# fPhân tích đặc trưng quan trọng
# Lấy tầm quan trọng của đặc trưng
feature_importances = final_model.feature_importances_

# Giả sử chúng ta có tên các đặc trưng
try:
    # Nếu X_train là DataFrame
    if hasattr(X_train, 'columns'):
        feature_names = X_train.columns.tolist()
    else:
        # Nếu không, tạo tên đặc trưng bằng số
        feature_names = [f"feature_{i}" for i in range(X_train.shape[1])]
except:
    feature_names = [f"feature_{i}" for i in range(X_train.shape[1])]

# Tạo DataFrame để trực quan hóa
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
})
importance_df = importance_df.sort_values('Importance', ascending=False)

# Top 20 đặc trưng quan trọng nhất
plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=importance_df.head(20))
plt.title('Top 20 đặc trưng quan trọng nhất')
plt.tight_layout()
plt.savefig('tabnet_feature_importance.png')
plt.show()